# **Topic Modeling and Search with Top2Vec :** 
# **Cable Car**

# **1. Import and Setup**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/My Drive/nlp')

## **a) To install Top2Vec library and pre-trained BERT sentence transformer options:**

In [3]:
# !pip install top2vec[sentence_transformers]
!pip install pynndescent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.9 MB/s 
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=c6bf2fe9280e1f949a0e345583325070bbe78d01e0a5d166f2874c539e811e83
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built pynndescent



## **b) Import Libraries**

In [4]:
from top2vec import Top2Vec 

import pandas as pd
import numpy as np

# **2. Import Dataset**

In [5]:
df = pd.read_excel("/content/gdrive/MyDrive/data/clean/formatted_reviews.xlsx")
df.sample(4)

,date,source,attraction,reviews,rating
919,2019.0,google_reviews,uss,selalu seru seperti biasanya mengenang masa pa...,10.0
1195,2019.0,google_reviews,uss,seruuuuu bukan pengen dateng lagi he,8.0
2065,2019.0,traveloka,uss,pertama kali ke singapura dan pertama kali ke ...,9.0
4373,2019.0,traveloka,sea_aquarium,sangat menyenangkan menghabiskan waktu di akhi...,9.0


In [6]:
df['date'] = df['date'].astype(int)

In [7]:
cable_df = df[df['attraction'] == 'cable_car']
cable_df = cable_df.sort_values(by=['date','rating'],ascending=[False,False])
# resetting index
cable_df = cable_df.reset_index(drop=True)
cable_df.tail(4)

,date,source,attraction,reviews,rating
173,2018,klook,cable_car,awesome view from the cable car really enjoy i...,10.0
174,2018,klook,cable_car,pertama kali naik singapore cable car kesannya...,10.0
175,2018,klook,cable_car,sangat menyenangkan jangan lupa untuk redeem d...,10.0
176,2018,klook,cable_car,kurang begitu menikmati saat kesingapore hujan...,8.0


In [8]:
cable_docs = cable_df.loc[:, "reviews"].astype(str).values.tolist()
cable_docs[:7]

['seru banget penukaran tiket juga sangat mudah ga perlu ribet untuk cetak cukup dari smartphone dan bisa langsung naik cable carnya rekomen banget beli di traveloka',
 'kerenn viewnya dan mau coba lagi lanjut',
 'bagus banget pemandangan dari atas terus bisa naik dan turun sepuasnya',
 'pemandangan sentosa island dari atas terlihat indah sekali yang bawa stroller bayi juga bisa masuk ke cable car jadi ga ribet kalau bawa bayi ada lift untuk ke atas khusus yang bawa bayi untuk aksesnya semua disiapin kalau di cable car singapura',
 'pemandangannya bagus sekali dari atas keren deh ngeringeri sedap naiknya karena tinggi sekali tapi pemandangannya juga bagus sekali dari atas lihat seluruh singapore',
 'gampang banget buat redeem tiket dan semuanya serba gampang jadi kita ngerasa nyaman pakai traveloka thank you',
 'seru sekaliii cukup sekali beli tiket bisa berkalikali menaikinya dalam sehari']

# **3. Create and Train N-Gram model (Bi-Gram and Tri-Gram)**
* Top2Vec uses Gensim **simple_preprocess** to do tokenization. 
* To create Trigrams and Bigrams, pass a customized tokenizer into Top2Vec tokenizer parameter. 
* Use Gensim to train this N-gram model. Documents passed to the Phrases function to inspect for N-grams text corpus.

In [9]:
import gensim
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags

# Build the bigram and trigram models 
sentence_stream = [doc.split(" ") for doc in cable_docs]
bigram = Phrases(sentence_stream, min_count=5, threshold=5, delimiter=b' ')
trigram = Phrases(bigram[sentence_stream], threshold=5, delimiter=b' ')

bigram_phraser = Phraser(bigram)
trigram_phraser = Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [10]:
# Create Bi-Grams
def bi_gram(doc):
    sentence_stream = simple_preprocess(strip_tags(str(doc)), deacc=True)
    return bigram_phraser[sentence_stream] 


# Create Bi-Grams and Tri-Grams
def tri_gram(doc):
    sentence_stream = simple_preprocess(strip_tags(str(doc)), deacc=True)
    gen_bigram = bigram_phraser[sentence_stream]
    gen_trigram = trigram_phraser[bigram_phraser[sentence_stream]]
    return gen_trigram

# **4. Train Top2Vec Model**

### **Parameters**:
* **Documents**: Input corpus, should be a list of strings.
* **Min_count**: (Optional, default 50) Ignores all words with total frequency lower than this. For smaller corpora a smaller min_count will be necessary.
* **Embedding_model** (string or callable) – The valid string options are: doc2vec , universal-sentence-encoder , universal-sentence-encoder-multilingual,distiluse-base-multilingual-cased , all-MiniLM-L6-v2 , paraphrase-multilingual-MiniLM-L12-v2 
* **tokenizer** (callable (Optional, default None)) – Override the default tokenization method. If None then gensim.utils.simple_preprocess will be used. ***Tokenizer must take a document and return a list of tokens***.

In [11]:
# FORM UNI-GRAMS, BI-GRAMS WITH BI-GRAM MODEL 

topicMODEL_bi = Top2Vec(documents= cable_docs, min_count=5, embedding_model = 'distiluse-base-multilingual-cased', tokenizer= bi_gram)  

2022-06-26 06:25:14,818 - top2vec - INFO - Pre-processing documents for training
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-06-26 06:25:14,880 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

2022-06-26 06:26:03,699 - top2vec - INFO - Creating joint document/word embedding
2022-06-26 06:26:18,287 - top2vec - INFO - Creating lower dimension embedding of documents
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-06-26 06:26:28,671 - top2vec - INFO - Finding dense areas of documents
2022-06-26 06:26:28,684 - top2vec - INFO - Finding topics


In [12]:
# FORM UNI-GRAMS, BI-GRAMS, TRI-GRAMS WITH TRI-GRAM MODEL 
 
topicMODEL_tri = Top2Vec(documents= cable_docs, min_count=5, embedding_model = 'distiluse-base-multilingual-cased', tokenizer=tri_gram)  

2022-06-26 06:26:28,706 - top2vec - INFO - Pre-processing documents for training
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-06-26 06:26:28,756 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model
2022-06-26 06:26:32,610 - top2vec - INFO - Creating joint document/word embedding
2022-06-26 06:26:46,966 - top2vec - INFO - Creating lower dimension embedding of documents
2022-06-26 06:26:50,760 - top2vec - INFO - Finding dense areas of documents
2022-06-26 06:26:50,771 - top2vec - INFO - Finding topics


In [13]:
# SAVE MODELS FOR FUTURE USE

topicMODEL_bi.save("/content/gdrive/MyDrive/data/models/bigram_cable_june26")

topicMODEL_tri.save("/content/gdrive/MyDrive/data/models/trigram_cable_june26")

In [ ]:
# LOAD MODEL FROM SAVED MODELS

topicMODEL_bi = Top2Vec.load("/content/gdrive/MyDrive/data/models/bigram_cable_june26")

topicMODEL_tri = Top2Vec.load("/content/gdrive/MyDrive/data/models/trigram_cable_june26")

# **5. Perform Topic Reduction** 

* Reduce the number of topics discovered by Top2Vec.
* The most representative topics of the corpus will be found, by iteratively merging each smallest topic to the most similar topic until num_topics is reached. 
* Get the hierarchy of reduced topics. The mapping of each original topic to the reduced topics is returned.

In [ ]:
# BI-GRAM MODEL TOPIC REDUCTION
topicMODEL_bi.hierarchical_topic_reduction(num_topics=10) 

topicMODEL_bi.get_topic_hierarchy()

In [ ]:
# TRI-GRAM MODEL TOPIC REDUCTION 
topicMODEL_tri.hierarchical_topic_reduction(num_topics=10) 

topicMODEL_tri.get_topic_hierarchy()

# **6. Explore Discovered Topics**

## **6a) Get Number of Topics**
This will return the number of topics that Top2Vec has found in the data.

In [14]:
# Show the Total Number of Topics 

print(f"Total Number of Topics generated (Topic Reduction Applied): \n")
print(f"Bi-gram Model : {topicMODEL_bi.get_num_topics(reduced=False)}")
print(f"Tri-gram Model : {topicMODEL_tri.get_num_topics(reduced=False)}")

Total Number of Topics generated (Topic Reduction Applied): 

Bi-gram Model : 2
Tri-gram Model : 2


## **6b) Get Topic Sizes**
This will return the number of documents most similar to each topic. Topics are in decreasing order of size.

Returns:
* topic_sizes: The number of documents most similar to each topic.
* topic_nums: The unique index of every topic will be returned.

### **6b.1. Bi-Gram and Tri-Gram Model Topic Sizes**

In [15]:
print("BI-GRAM MODEL where N = 1 and 2")
print("================================\n")
topic_sizes_bi, topic_nums_bi = topicMODEL_bi.get_topic_sizes(reduced=False) 

print(f"Unique index numbers of every topic: {topic_nums_bi} ")
print()
print(f"Number of documents for each unique topic: {topic_sizes_bi} \n")

print("TRI-GRAM MODEL where N = 1, 2 and 3")
print("====================================\n")
topic_sizes_tri, topic_nums_tri = topicMODEL_tri.get_topic_sizes(reduced=False) 
print(f"Unique index numbers of every topic: {topic_nums_tri} ")
print()
print(f"Number of documents for each unique topic: {topic_sizes_tri} ") 

BI-GRAM MODEL where N = 1 and 2

Unique index numbers of every topic: [0 1] 

Number of documents for each unique topic: [93 84] 

TRI-GRAM MODEL where N = 1, 2 and 3

Unique index numbers of every topic: [0 1] 

Number of documents for each unique topic: [104  73] 


## **6c) Get Topics**
This will return the topics in decreasing size.

Returns:

* topic_words: For each topic the top 50 words are returned, in decreasing order of semantic similarity to topic.

* word_scores: For each topic the cosine similarity scores (in decreasing order) of the top 50 words to the topic are returned.

* topic_nums: The unique index of every topic will be returned.

### **c.1.  BI-GRAM Model Topics**

In [16]:
print(f"TOPICS FOR BI-GRAM MODEL: \n")
topic_words_bi, word_scores_bi, topic_nums_bi = topicMODEL_bi.get_topics(topicMODEL_bi.get_num_topics(reduced=False))

for count,(words_bi, scores_bi, num_bi) in enumerate(zip(topic_words_bi, word_scores_bi, topic_nums_bi)):
    print(f"{count+1}. Topic {num_bi+1}") 
    print(f"Topic Words : \n{words_bi} \n")
    # print(f"Cosine Similarity :\n{scores_bi}\n")

TOPICS FOR BI-GRAM MODEL: 

1. Topic 1
Topic Words : 
['tiketnya' 'redeem tiket' 'tiket' 'cable car' 'lebih murah' 'naik cable'
 'line' 'trip' 'di traveloka' 'station' 'stasiun' 'sangat membantu'
 'harga lebih' 'mudah' 'murah' 'traveloka' 'membeli' 'car'
 'sangat menyenangkan' 'hanya' 'praktis' 'harga' 'sangat' 'bagus'
 'voucher' 'indah' 'singapore' 'barcode' 'untuk' 'terima kasih' 'klook'
 'karena' 'jadi' 'beli di' 'langsung' 'melalui' 'sih' 'lumayan'
 'singapura' 'gak' 'mount faber' 'menyenangkan' 'di' 'banyak' 'saja'
 'cukup' 'harus' 'senang' 'ini' 'dan'] 

2. Topic 2
Topic Words : 
['cable car' 'naik cable' 'car' 'line' 'station' 'sangat menyenangkan'
 'stasiun' 'trip' 'indah' 'di traveloka' 'traveloka' 'menyenangkan'
 'mount faber' 'bagus' 'view' 'singapore' 'sangat membantu' 'melalui'
 'dari ketinggian' 'melihat pemandangan' 'singapura' 'the' 'ketinggian'
 'bisa melihat' 'untuk' 'pengalaman yang' 'murah' 'pengalaman' 'tiketnya'
 'klook' 'keren' 'membeli' 'senang' 'pemandangan' 'v

### **c.2. Tri-Gram Model Topics**

In [17]:
print(f"TOPICS FOR TRI-GRAM MODEL : \n")
topic_words_tri, word_scores_tri, topic_nums_tri = topicMODEL_tri.get_topics(topicMODEL_tri.get_num_topics(reduced=False))

for count, (words_tri, scores_tri, num_tri) in enumerate(zip(topic_words_tri, word_scores_tri, topic_nums_tri)):
    print(f"{count+1}. Topic {num_tri+1}") 
    print(f"Topic Words : \n{words_tri}\n")
    # print(f"Cosine Similarity :\n{scores_tri}\n")

TOPICS FOR TRI-GRAM MODEL : 

1. Topic 1
Topic Words : 
['naik cable car' 'cable car' 'line' 'station' 'stasiun'
 'sangat menyenangkan' 'di traveloka' 'trip' 'singapore'
 'terima kasih traveloka' 'indah' 'singapura' 'tiketnya' 'mount faber'
 'traveloka' 'pengalaman yang' 'redeem tiket' 'membeli' 'dari ketinggian'
 'pengalaman' 'harga lebih murah' 'terima kasih klook' 'menyenangkan'
 'bagus' 'untuk' 'sangat membantu' 'lebih murah' 'melihat pemandangan'
 'melalui' 'ketinggian' 'bisa melihat' 'view' 'the' 'karena' 'murah'
 'pertama' 'klook' 'terima kasih' 'panjang' 'beli di' 'senang' 'sangat'
 'jadi' 'yang' 'di' 'ini' 'pemandangannya' 'barcode' 'tiket'
 'sentosa island']

2. Topic 2
Topic Words : 
['tiketnya' 'redeem tiket' 'tiket' 'harga lebih murah' 'lebih murah'
 'trip' 'naik cable car' 'terima kasih traveloka' 'di traveloka'
 'sangat membantu' 'cable car' 'line' 'mudah' 'traveloka' 'murah'
 'station' 'membeli' 'stasiun' 'terima kasih klook' 'barcode'
 'sangat menyenangkan' 'praktis' '

## **6d) Search Documents by Topic**
We are going to search by topic; for example Topic 3. 

Returns:

* documents:  The documents in a list, the most similar are first.
* doc_scores:  Semantic similarity of document to topic. The cosine similarity of the document and topic vector.
* doc_ids:  Unique ids of documents. If ids were not given, the index of document in the original corpus.

For each of the returned documents we are going to print its content, score and document number.

### **d.1. Search Documents By Topic Number Using Bi-Gram Model**

In [18]:
# SEARCH DOCUMENTS BY TOPIC NUMBER USING BI-GRAM MODEL 
topic_num_bi = 0
documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= 5, reduced=False) 

print(f"TOPIC NUMBER: {topic_num_bi}\n")
for count, (doc_bi, docscore_bi, doc_id_bi) in enumerate(zip(documents_bi, document_scores_bi, document_ids_bi)):
    print(f"{count}. Document: {doc_id_bi},  Score: {docscore_bi}")
    print("-----------")
    print(doc_bi)
    print("-----------\n")

TOPIC NUMBER: 0

0. Document: 23,  Score: 0.6480955481529236
-----------
dg tiket yang dibeli di traveloka hanya perlu menunjukkannya kepada petugas loket maka langsung diberi tiket pass untuk naik cable car sangat mudah terima kasih oiya tips bagi traveler yang lain jangan lupa untuk diprint iya tiketnya
-----------

1. Document: 21,  Score: 0.6341419816017151
-----------
pesan tiket di traveloka lebih murah aku pesan saat di depan loket tiket dan langsung diredeem gak pakai ribet tiket bisa digunakan untuk 2 line sentosa line dan mount faber line
-----------

2. Document: 162,  Score: 0.6264634132385254
-----------
beli di traveloka lebih murah redeem tiket langsung di loket cable carnya apabila membeli tiket sky pass harap tiketnya disimpan sampai selesai menggunakan wahana tsb karena di setiap stasiun tiket akan dicek dan dicentang apabila tiket hilang dan kita belum menyelesaikan perjalanan maka kita harus membeli kembali tiketnya
-----------

3. Document: 146,  Score: 0.590809464

### **d.2. Search Documents By Topic Number Using Tri-Gram Model**

In [19]:
# SEARCH DOCUMENTS BY TOPIC NUMBER USING TRI-GRAM MODEL 
topic_num_tri = 1
documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri, num_docs= 5, reduced=False) 
print(f"TOPIC NUMBER: {topic_num_bi}\n")
for count, (doc_tri, docscore_tri, doc_id_tri) in enumerate(zip(documents_tri, document_scores_tri, document_ids_tri)):
    print(f"{count}. Document: {doc_id_tri},  Score: {docscore_tri}")
    print("-----------")
    print(doc_tri)
    print("-----------\n")

TOPIC NUMBER: 0

0. Document: 23,  Score: 0.6055053472518921
-----------
dg tiket yang dibeli di traveloka hanya perlu menunjukkannya kepada petugas loket maka langsung diberi tiket pass untuk naik cable car sangat mudah terima kasih oiya tips bagi traveler yang lain jangan lupa untuk diprint iya tiketnya
-----------

1. Document: 21,  Score: 0.6028358936309814
-----------
pesan tiket di traveloka lebih murah aku pesan saat di depan loket tiket dan langsung diredeem gak pakai ribet tiket bisa digunakan untuk 2 line sentosa line dan mount faber line
-----------

2. Document: 58,  Score: 0.5931907892227173
-----------
sangat mudah mendapatkan tiket fisiknya karena klook tinggal scan langsung dapat deh tiketnya
-----------

3. Document: 80,  Score: 0.5835537910461426
-----------
harga tiket hanya untuk round trip di loketnya pun tidak menjual tiket single trip evoucher sangat membantu karena redemption menggunakan jalur khusus
-----------

4. Document: 162,  Score: 0.5674034357070923
----

# **7) Constructing Dataframe to Collate Information of Key Topics**

## **7a) BI-GRAM MODEL**

In [20]:
# CONSTRUCT DATAFRAME OF BI-GRAM MODEL WITH KEY INFORMATION FOR EXPORT
# initialize data of lists.
bigram_table = {
              'topic number': [num_bi for num_bi in topic_nums_bi],
              'topic words': [words_bi for words_bi in topic_words_bi],
              'cosine similarity metrics': [scores_bi for scores_bi in word_scores_bi],
              'similar documents': [size_bi for size_bi in topic_sizes_bi],
               }
 
# Create DataFrame
bigram_df = pd.DataFrame(bigram_table)
bigram_df['topic number'] = bigram_df['topic number'].apply(lambda x: x + 1)

# print(bigram_df.head(2))
bigram_df.tail()

,topic number,topic words,cosine similarity metrics,similar documents
0,1,"[tiketnya, redeem tiket, tiket, cable car, leb...","[0.43573993, 0.4087272, 0.2481237, 0.2307407, ...",93
1,2,"[cable car, naik cable, car, line, station, sa...","[0.63322246, 0.4604483, 0.30241674, 0.23955518...",84


### **7a.1. Build Functions to Expand Column Features in Bi-Gram Dataframe**

In [21]:
def get_document_index(topic_num_bi, num_doc):
    documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= num_doc, reduced=False)
    document_index = [doc_id_bi for doc_id_bi in document_ids_bi]
    return document_index 

def get_document_scores(topic_num_bi, num_doc):
    documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= num_doc, reduced=False)
    document_scores = [docscore_bi for docscore_bi in document_scores_bi]
    return document_scores


def get_documents(topic_num_bi, num_doc):
    documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= num_doc, reduced=False)
    actual_document = [doc_bi for doc_bi in documents_bi]
    return actual_document


In [22]:
bigram_df['document index'] = bigram_df['topic number'].apply(lambda x: get_document_index(x-1, bigram_df['similar documents'].values[x-1]))

bigram_df['document scores'] = bigram_df['topic number'].apply(lambda x: get_document_scores(x-1, bigram_df['similar documents'].values[x-1]))

bigram_df['actual reviews'] = bigram_df['topic number'].apply(lambda x: get_documents(x-1, bigram_df['similar documents'].values[x-1]))

bigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews
0,1,"[tiketnya, redeem tiket, tiket, cable car, leb...","[0.43573993, 0.4087272, 0.2481237, 0.2307407, ...",93,"[23, 21, 162, 146, 80, 119, 154, 58, 175, 170,...","[0.64809555, 0.634142, 0.6264634, 0.59080946, ...",[dg tiket yang dibeli di traveloka hanya perlu...
1,2,"[cable car, naik cable, car, line, station, sa...","[0.63322246, 0.4604483, 0.30241674, 0.23955518...",84,"[145, 111, 65, 173, 155, 121, 136, 163, 36, 10...","[0.644899, 0.586964, 0.5862599, 0.58275795, 0....",[pemandangan bagus sekali dari atas cable car ...


In [23]:
# for count, (index, review) in enumerate(zip(bigram_df['document index'][28], bigram_df['actual reviews'][28])):
#   print(f"{count}: {index} = {review})")

bigram_df.to_excel(f'/content/gdrive/MyDrive/data/export/bigram_df_cable.xlsx', index=False)

## **7b) TRI-GRAM MODEL**

In [24]:
# CONSTRUCT DATAFRAME CONTAINING KEY INFORMATION FOR EXPORT
# initialize data of lists.
trigram_table = {
                'topic number': [num_tri for num_tri in topic_nums_tri],
                'topic words': [words_tri for words_tri in topic_words_tri],
                'cosine similarity metrics': [scores_tri for scores_tri in word_scores_tri],
                'similar documents': [size_tri for size_tri in topic_sizes_tri],
                 }

# Create DataFrame
trigram_df = pd.DataFrame(trigram_table)

trigram_df['topic number'] = trigram_df['topic number'].apply(lambda x: x + 1)
trigram_df.tail()

,topic number,topic words,cosine similarity metrics,similar documents
0,1,"[naik cable car, cable car, line, station, sta...","[0.6052697, 0.59935784, 0.25098515, 0.15974963...",104
1,2,"[tiketnya, redeem tiket, tiket, harga lebih mu...","[0.45234329, 0.42621398, 0.25485027, 0.1973314...",73


### **7b.1. Build Functions to Expand Column Features in Tri-Gram Dataframe**

In [25]:
def get_document_index_tri(topic_num_tri, num_docs):
    documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri , num_docs= num_docs, reduced=False)
    document_index = [doc_id_tri for doc_id_tri in document_ids_tri]
    return document_index 

def get_document_scores_tri(topic_num_tri, num_docs):
    documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri , num_docs= num_docs, reduced=False)
    document_scores = [docscore_tri for docscore_tri in document_scores_tri]
    return document_scores


def get_documents_tri(topic_num_tri, num_docs):
    documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri , num_docs= num_docs, reduced=False)
    actual_document = [doc_tri for doc_tri in documents_tri]
    return actual_document


In [26]:
trigram_df['document index'] = trigram_df['topic number'].apply(lambda x: get_document_index_tri(x-1, trigram_df['similar documents'].values[x-1]))

trigram_df['document scores'] = trigram_df['topic number'].apply(lambda x: get_document_scores_tri(x-1, trigram_df['similar documents'].values[x-1]))

trigram_df['actual reviews'] = trigram_df['topic number'].apply(lambda x: get_documents_tri(x-1, trigram_df['similar documents'].values[x-1]))

trigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews
0,1,"[naik cable car, cable car, line, station, sta...","[0.6052697, 0.59935784, 0.25098515, 0.15974963...",104,"[76, 154, 122, 173, 136, 13, 174, 145, 115, 11...","[0.6163023, 0.6077281, 0.5988513, 0.58744514, ...",[beli tiket round trip cable car satu kali bis...
1,2,"[tiketnya, redeem tiket, tiket, harga lebih mu...","[0.45234329, 0.42621398, 0.25485027, 0.1973314...",73,"[23, 21, 58, 80, 162, 146, 59, 119, 0, 106, 17...","[0.60550535, 0.6028359, 0.5931908, 0.5835538, ...",[dg tiket yang dibeli di traveloka hanya perlu...


In [27]:
# for count, (index, review) in enumerate(zip(trigram_df['document index'][0], trigram_df['actual reviews'][0])):
#   print(f"{count}: {index} = {review})")

trigram_df.to_excel(f'/content/gdrive/MyDrive/data/export/trigram_df_cable.xlsx', index=False)

# **8. Defining Topic Labels on Reviews** 
## **8a) User-Define Topic Labels** 
### **8a.1:  BI-GRAM Model** 

In [28]:
# Create the dictionary containing the data of the new column
bigram_col_dict = { 'Topic 1': 'visitors experiences',                 'Topic 2': 'features visitors like',
                  
                  }
                   
# Assign the values of the dictionary as the values of the new column
bigram_df['topic label'] = bigram_col_dict.values()
bigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews,topic label
0,1,"[tiketnya, redeem tiket, tiket, cable car, leb...","[0.43573993, 0.4087272, 0.2481237, 0.2307407, ...",93,"[23, 21, 162, 146, 80, 119, 154, 58, 175, 170,...","[0.64809555, 0.634142, 0.6264634, 0.59080946, ...",[dg tiket yang dibeli di traveloka hanya perlu...,visitors experiences
1,2,"[cable car, naik cable, car, line, station, sa...","[0.63322246, 0.4604483, 0.30241674, 0.23955518...",84,"[145, 111, 65, 173, 155, 121, 136, 163, 36, 10...","[0.644899, 0.586964, 0.5862599, 0.58275795, 0....",[pemandangan bagus sekali dari atas cable car ...,features visitors like


In [29]:
# CREATE COPY OF ORIGINAL DATAFRAME
cable_DF_BI = cable_df.copy()
cable_DF_BI.tail(3)

,date,source,attraction,reviews,rating
174,2018,klook,cable_car,pertama kali naik singapore cable car kesannya...,10.0
175,2018,klook,cable_car,sangat menyenangkan jangan lupa untuk redeem d...,10.0
176,2018,klook,cable_car,kurang begitu menikmati saat kesingapore hujan...,8.0


In [30]:
print([i for i in range(topicMODEL_bi.get_num_topics(reduced=False))])

[0, 1]


In [31]:
# FUNCTION TO CONNECT TOPIC LABELS TO INDEX LIST
def get_indexLIST(i):
      topic = [doc_ind for doc_ind in bigram_df['document index'][i]]
      return topic

In [32]:
for i in range(topicMODEL_bi.get_num_topics(reduced=False)):
    cable_DF_BI.loc[get_indexLIST(i), 'topic label'] = bigram_df['topic label'][i]
    

In [33]:
# FUNCTION TO CONNECT DOCUMENT COSINE SCORE TO INDEX LIST
def get_scores(i):
    score = [doc_score for doc_score in bigram_df['document scores'][i]]
    return score

In [34]:
for i in range(topicMODEL_bi.get_num_topics(reduced=False)):
    cable_DF_BI.loc[get_indexLIST(i), 'cosine score review/topic'] = get_scores(i) 

In [35]:
cable_DF_BI.reset_index(drop=True)
cable_DF_BI

,date,source,attraction,reviews,rating,topic label,cosine score review/topic
0,2020,traveloka,cable_car,seru banget penukaran tiket juga sangat mudah ...,10.0,visitors experiences,0.549170
1,2020,traveloka,cable_car,kerenn viewnya dan mau coba lagi lanjut,10.0,features visitors like,0.065904
2,2020,traveloka,cable_car,bagus banget pemandangan dari atas terus bisa ...,10.0,features visitors like,0.153755
3,2020,traveloka,cable_car,pemandangan sentosa island dari atas terlihat ...,10.0,features visitors like,0.456947
4,2020,traveloka,cable_car,pemandangannya bagus sekali dari atas keren de...,10.0,features visitors like,0.201371
...,...,...,...,...,...,...,...
172,2018,klook,cable_car,kebetulan menaiki cable car saat weekday jadi ...,10.0,features visitors like,0.447057
173,2018,klook,cable_car,awesome view from the cable car really enjoy i...,10.0,features visitors like,0.582758
174,2018,klook,cable_car,pertama kali naik singapore cable car kesannya...,10.0,visitors experiences,0.474933
175,2018,klook,cable_car,sangat menyenangkan jangan lupa untuk redeem d...,10.0,visitors experiences,0.553335


In [37]:
cable_DF_BI.isnull().values.any()

False

In [39]:
cable_DF_BI.to_excel(f'/content/gdrive/MyDrive/data/export/cable_DF_BI.xlsx', index=False)

### **8a.2  TRI-GRAM Model** 

In [40]:
# Create the dictionary containing the data of the new column
trigram_col_dict = { 'Topic 1': 'visitors experiences',                 'Topic 2': 'ticketing',
                  
                  }
   
# Assign the values of the dictionary as the values of the new column
trigram_df['topic label'] = trigram_col_dict.values()
trigram_df.tail(3)

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews,topic label
0,1,"[naik cable car, cable car, line, station, sta...","[0.6052697, 0.59935784, 0.25098515, 0.15974963...",104,"[76, 154, 122, 173, 136, 13, 174, 145, 115, 11...","[0.6163023, 0.6077281, 0.5988513, 0.58744514, ...",[beli tiket round trip cable car satu kali bis...,visitors experiences
1,2,"[tiketnya, redeem tiket, tiket, harga lebih mu...","[0.45234329, 0.42621398, 0.25485027, 0.1973314...",73,"[23, 21, 58, 80, 162, 146, 59, 119, 0, 106, 17...","[0.60550535, 0.6028359, 0.5931908, 0.5835538, ...",[dg tiket yang dibeli di traveloka hanya perlu...,ticketing


In [41]:
# CREATE COPY OF ORIGINAL DATAFRAME
cable_DF_TRI = cable_df.copy()
cable_DF_TRI.tail(3)

,date,source,attraction,reviews,rating
174,2018,klook,cable_car,pertama kali naik singapore cable car kesannya...,10.0
175,2018,klook,cable_car,sangat menyenangkan jangan lupa untuk redeem d...,10.0
176,2018,klook,cable_car,kurang begitu menikmati saat kesingapore hujan...,8.0


In [42]:
print([i for i in range(topicMODEL_tri.get_num_topics(reduced=False))])

[0, 1]


In [43]:
# FUNCTION TO CONNECT TOPIC LABELS TO INDEX LIST
def get_indexLIST(i):
      topic = [doc_ind for doc_ind in trigram_df['document index'][i]]
      return topic

In [44]:
for i in range(topicMODEL_tri.get_num_topics(reduced=False)):
    cable_DF_TRI.loc[get_indexLIST(i), 'topic_label'] = trigram_df['topic label'][i]

In [45]:
# FUNCTION TO CONNECT DOCUMENT COSINE SCORE TO INDEX LIST
def get_scores(i):
    score = [doc_score for doc_score in trigram_df['document scores'][i]]
    return score

In [46]:
for i in range(topicMODEL_tri.get_num_topics(reduced=False)):
    cable_DF_TRI.loc[get_indexLIST(i), 'cosine score review/topic'] = get_scores(i) 

In [48]:
cable_DF_TRI.reset_index(drop=True)
cable_DF_TRI

,date,source,attraction,reviews,rating,topic_label,cosine score review/topic
0,2020,traveloka,cable_car,seru banget penukaran tiket juga sangat mudah ...,10.0,ticketing,0.536280
1,2020,traveloka,cable_car,kerenn viewnya dan mau coba lagi lanjut,10.0,visitors experiences,0.041594
2,2020,traveloka,cable_car,bagus banget pemandangan dari atas terus bisa ...,10.0,visitors experiences,0.113179
3,2020,traveloka,cable_car,pemandangan sentosa island dari atas terlihat ...,10.0,visitors experiences,0.527580
4,2020,traveloka,cable_car,pemandangannya bagus sekali dari atas keren de...,10.0,visitors experiences,0.251087
...,...,...,...,...,...,...,...
172,2018,klook,cable_car,kebetulan menaiki cable car saat weekday jadi ...,10.0,visitors experiences,0.411528
173,2018,klook,cable_car,awesome view from the cable car really enjoy i...,10.0,visitors experiences,0.587445
174,2018,klook,cable_car,pertama kali naik singapore cable car kesannya...,10.0,visitors experiences,0.574220
175,2018,klook,cable_car,sangat menyenangkan jangan lupa untuk redeem d...,10.0,ticketing,0.523038


In [49]:
cable_DF_TRI.isnull().values.any()

False

In [50]:
cable_DF_TRI.to_excel(f'/content/gdrive/MyDrive/data/export/cable_DF_TRI.xlsx', index=False)

# **9.  Visualization**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
uss_topred = pd.read_excel("/content/gdrive/MyDrive/data/export/uss_DF_BIreduced.xlsx")
uss_topred.isnull().values.any()
uss_topred = uss_topred.loc[uss_topred['date'] != 2017]
uss_topred

In [ ]:
sns.color_palette("gist_rainbow")

In [ ]:
sns.set_palette("gist_rainbow")
sns.set_theme(style="ticks", font_scale=1.4)

In [ ]:
plt.figure(figsize=(18,10))

hue_order = [2022, 2021, 2020, 2019, 2018]
sns.histplot(data=uss_topred, y="topic_label", hue="date", hue_order=hue_order, multiple="stack", bins= 50, palette="Set1") 


In [ ]:
sns.color_palette('Set3')
plt.figure(figsize=(25,10))

hue_order = [2018, 2019, 2020,2021, 2022]
sns.histplot(data=uss_topred, x="topic_label", hue="date", hue_order=hue_order, multiple="dodge", bins=100, palette="Set1") 


In [ ]:
plt.figure(figsize=(16,11))

sns.countplot(data=uss_topred, y="topic_label", hue="date", dodge=False)